In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "/share/dutta/eyao/dataset/kaggle/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/share/dutta/eyao/dataset/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory () that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder


# Load the training data
train_data = pd.read_csv('/share/dutta/eyao/dataset/kaggle/spaceship-titanic/train.csv')

# Data Preprocessing
train_data.fillna(0, inplace=True)  # Replace NaN values with 0 for simplicity

# Convert categorical variables to numerical representations
le = LabelEncoder()
train_data['CryoSleep'] = le.fit_transform(train_data['CryoSleep'])
# Repeat the above step for other categorical variables if needed

# Feature selection
features = ['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
X = train_data[features]
y = train_data['Transported']

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)

# Build a small neural network model
model = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_valid, y_valid))

# Model Evaluation on Validation Set
y_pred_prob = model.predict(X_valid)
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions
accuracy = accuracy_score(y_valid, y_pred)
print(f"Validation Accuracy: {accuracy}")

# Predictions on Test Set
test_data = pd.read_csv('/share/dutta/eyao/dataset/kaggle/spaceship-titanic/test.csv')
# Perform the same data preprocessing steps on the test data
test_data.fillna(0, inplace=True)
test_data['CryoSleep'] = le.transform(test_data['CryoSleep'])
# Repeat the above steps for other categorical variables if needed

X_test = test_data[features]
X_test = scaler.transform(X_test)

# Make predictions
test_predictions_prob = model.predict(X_test)
test_predictions = (test_predictions_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

# Prepare submission file
submission = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Transported': test_predictions.flatten()})
submission.to_csv('submission_small_neural_network.csv', index=False)
